In [18]:
from Indicator import Indicator 
import random
import glob
import numpy as np
import pandas as pd


datalist = [n for n in glob.glob('Dataset/*.csv')]
indicator = Indicator()
data = pd.read_csv(datalist[0],index_col = False)

In [38]:
def initial_population(n):#number Population
    pop = []
    for i in range(n):
        pop.append([[random.randint(1,60),random.randint(1,60),random.randint(1,60)],0])
    return pop

def cal_fitness(pop):
    calpop = []
    for cm in pop:
        fitness = 0
        # df = pd.read_csv(datalist[0],index_col = False)
        df = data
        df = indicator.MACD(df,cm[0][0],cm[0][1],cm[0][2])
        df['signal']  = np.where((df['MACD']>df['SIGNAL LINE'])&(df['MACD'].shift()<df['SIGNAL LINE'].shift()),'buy',
                                  (np.where((df['MACD'] < df['SIGNAL LINE'])  & (df['MACD'].shift()>df['SIGNAL LINE'].shift()),'sell','wait or hold')))
        
        signal = []
        for index,row in df.iterrows():
            if row['signal'] == 'buy':
                 signal.append(['buy',row['Close']])        
            elif row['signal'] == 'sell':
                 signal.append(['sell',row['Close']])
        
        if not signal:
            calpop.append([cm[0],0])
        else:
            if signal[0][0]=='sell':
                del signal[0]
            if len(signal) %2!=0:
                del signal[-1]

            money = 100000
            b = []
            s = []
            MACD = []
            for i in signal:
                if i[0] =='buy':
                    b.append(i[1])
                else:
                    s.append(i[1])
            macdbuy = len(b)
            macdsell = len(s)
            pfmacd= []
            profitmacd = []
            for i in range(len(b)):
                pfmacd.append(((s[i]-b[i])/(b[i]))*money)
                profitmacd.append(((s[i]-b[i])/(b[i])))
            pfmacd = sum(pfmacd)
            calpop.append([cm[0],pfmacd])
    return calpop
        
def binarytournament(p):
    a = random.randint(0,len(p)-1)
    b = random.randint(0,len(p)-1)
    while a==b:
        b = random.randint(0,len(p)-1)
    
    pr1,pr2 = p[a],p[b]
          
    if pr1[1] > pr2[1] :    
        return pr1
    else:
        return pr2

def crossover(p1,p2):
    pcent = random.randint(0,len(p1[0])-1)
    newp1,newp2 = [],[]
    newp1 = p1[0][pcent:]+p2[0][:pcent]
    newp2 = p2[0][pcent:]+p1[0][:pcent]

    p1[0] ,p2[0]= newp1,newp2
    return p1,p2

def mutation(o):
  Mutarate = (1/len(o[0]))*100 # mutation rate = (1/individual size) * 100 

  for i in range(len(o[0])):
    if random.randint(0,100) <= int(Mutarate):
      o[0][i] = random.randint(1,60)

  return o
        


In [39]:
#input
popsize = 30

#variable
generation =1
Maxgen = 200
Elite = 10

#First Generation
pop = initial_population(popsize)
pop[0] = [[12,26,9],0]
print(pop)
pop = cal_fitness(pop)
pop = sorted(pop,key=lambda l:l[1],reverse=True)
print(pop)

[[[12, 26, 9], 0], [[42, 4, 11], 0], [[40, 29, 9], 0], [[42, 54, 41], 0], [[37, 14, 44], 0], [[60, 59, 42], 0], [[47, 18, 48], 0], [[2, 39, 57], 0], [[27, 41, 16], 0], [[23, 11, 36], 0], [[38, 19, 38], 0], [[15, 3, 50], 0], [[32, 29, 57], 0], [[14, 51, 23], 0], [[12, 46, 48], 0], [[25, 27, 45], 0], [[46, 12, 17], 0], [[38, 52, 30], 0], [[10, 4, 24], 0], [[5, 26, 8], 0], [[40, 40, 42], 0], [[25, 27, 24], 0], [[45, 43, 13], 0], [[53, 58, 36], 0], [[34, 56, 37], 0], [[52, 12, 41], 0], [[4, 37, 20], 0], [[6, 39, 27], 0], [[31, 54, 42], 0], [[17, 60, 5], 0]]
[[[31, 54, 42], 252429.4953702356], [[34, 56, 37], 239292.08197927554], [[38, 52, 30], 225736.10492769067], [[53, 58, 36], 221486.35536489927], [[42, 54, 41], 216884.60236702472], [[25, 27, 45], 209939.61727811652], [[27, 41, 16], 189079.3823793492], [[12, 46, 48], 184889.07320485124], [[25, 27, 24], 171728.66843942716], [[14, 51, 23], 170973.34584507436], [[2, 39, 57], 158897.66655419438], [[4, 37, 20], 146487.11519916443], [[17, 60, 5

In [40]:
# random.seed(0)
while generation < Maxgen:

    newpop = []
    for i in range(int(len(pop)*Elite/100)):
        newpop.append(pop[i])
        del pop[i]

    while len(newpop) < popsize:
        parent1,parent2 = binarytournament(pop),binarytournament(pop)
        while parent1 == parent2:
                parent2 = binarytournament(pop)
        # print(parent1)
        parent1,parent2 = crossover(parent1,parent2)
        parent1,parent2 = mutation(parent1),mutation(parent2)
    
        newpop.append(parent1)
        newpop.append(parent2)
    print('Gen =',generation,'Max Fitness =',newpop[0][1])
    generation += 1
    pop = newpop
    pop = cal_fitness(pop)
    pop = sorted(pop,key=lambda l:l[1],reverse=True)


Gen = 1 Max Fitness = 252429.4953702356
Gen = 2 Max Fitness = 252429.4953702356
Gen = 3 Max Fitness = 252429.4953702356
Gen = 4 Max Fitness = 252429.4953702356
Gen = 5 Max Fitness = 252429.4953702356
Gen = 6 Max Fitness = 252429.4953702356
Gen = 7 Max Fitness = 252429.4953702356
Gen = 8 Max Fitness = 252429.4953702356
Gen = 9 Max Fitness = 252429.4953702356
Gen = 10 Max Fitness = 252429.4953702356
Gen = 11 Max Fitness = 252429.4953702356
Gen = 12 Max Fitness = 252429.4953702356
Gen = 13 Max Fitness = 252429.4953702356
Gen = 14 Max Fitness = 252429.4953702356
Gen = 15 Max Fitness = 252429.4953702356
Gen = 16 Max Fitness = 252429.4953702356
Gen = 17 Max Fitness = 252429.4953702356
Gen = 18 Max Fitness = 252429.4953702356
Gen = 19 Max Fitness = 252429.4953702356
Gen = 20 Max Fitness = 252429.4953702356
Gen = 21 Max Fitness = 252429.4953702356
Gen = 22 Max Fitness = 252429.4953702356
Gen = 23 Max Fitness = 252791.81050448664
Gen = 24 Max Fitness = 252791.81050448664
Gen = 25 Max Fitness = 

In [37]:
pop

[[[18, 59, 50], 262067.91334935167],
 [[35, 51, 43], 258295.69081377034],
 [[32, 37, 53], 227219.5041458485],
 [[31, 47, 14], 219543.0220373353],
 [[3, 59, 58], 214977.29332261457],
 [[48, 56, 57], 186456.46564476035],
 [[44, 57, 14], 184538.43189997022],
 [[44, 57, 14], 184538.43189997022],
 [[53, 57, 9], 183097.88581977208],
 [[53, 57, 9], 183097.88581977208],
 [[53, 57, 9], 183097.88581977208],
 [[46, 49, 19], 171813.1796323668],
 [[27, 50, 33], 171422.19134515207],
 [[27, 50, 33], 171422.19134515207],
 [[3, 46, 23], 150763.31533982058],
 [[3, 46, 23], 150763.31533982058],
 [[3, 46, 23], 150763.31533982058],
 [[12, 25, 19], 127423.58596828242],
 [[18, 56, 8], 127313.05795035348],
 [[8, 20, 51], 123784.81908677403],
 [[8, 20, 51], 123784.81908677403],
 [[8, 20, 51], 123784.81908677403],
 [[8, 20, 51], 123784.81908677403],
 [[3, 59, 41], 122504.44147626724],
 [[26, 4, 4], 118396.66741734356],
 [[44, 3, 13], 81103.34154411514],
 [[49, 28, 6], 62629.61764020024],
 [[17, 10, 49], 49970.7

In [41]:
pop

[[[36, 46, 44], 261968.8850453576],
 [[19, 59, 23], 238780.53108526845],
 [[36, 37, 44], 227068.8008282375],
 [[35, 45, 55], 226572.15379098264],
 [[35, 45, 55], 226572.15379098264],
 [[35, 45, 55], 226572.15379098264],
 [[24, 26, 47], 210694.73005333828],
 [[24, 26, 47], 210694.73005333828],
 [[37, 50, 11], 189517.56956869934],
 [[42, 50, 24], 183773.40364513965],
 [[26, 47, 26], 181422.21923138137],
 [[31, 13, 6], 89309.36066652807],
 [[31, 13, 6], 89309.36066652807],
 [[31, 13, 6], 89309.36066652807],
 [[31, 13, 6], 89309.36066652807],
 [[10, 1, 52], 83045.5119899626],
 [[10, 1, 52], 83045.5119899626],
 [[37, 8, 32], 52300.272303794285],
 [[32, 1, 23], 50782.72249181206],
 [[32, 1, 23], 50782.72249181206],
 [[43, 13, 25], 34014.910657067914],
 [[43, 13, 25], 34014.910657067914],
 [[55, 8, 60], 18690.016663340983],
 [[55, 8, 60], 18690.016663340983],
 [[40, 1, 25], 4154.219859660753],
 [[37, 37, 19], 0],
 [[49, 12, 32], -14449.874353981411],
 [[49, 12, 32], -14449.874353981411],
 [[4